# **London Bike Rides EDA**
by [Gaurang  Acharekar](https://www.linkedin.com/in/gaurang-acharekar/)<br>



Dataset : [London Bike Sharing Dataset](https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset/data)

---


**Goal:** Transforming raw urban mobility data into actionable insights through automated preprocessing and data enrichment for Tableau visualization.

## **Technical Skill Summary**

- **Data Wrangling:** Handled type conversions and schema standardization for over 17,000 records.
- **Normalization:** Engineered percentage-based humidity scales for consistent statistical comparison.
- **Semantic Mapping:** Decoded legacy integer classifications into human-readable categorical data.
- **Tableau Integration:** Optimized final data structures specifically for Tableau Representation.

## **Table of Contents**
1. [Data Acquisition & Initial Audit](#1)
2. [Schema Standardization (Renaming)](#2)
3. [Feature Engineering & Transformation](#3)
4. [Final Export for Visualization](#4)

<a id="1"></a>

## **1. Data Acquisition & Initial Audit**

Before performing any transformations, it is critical to understand the grain of the data and identify any structural issues. This step ensures the dataset is complete and helps determine the necessary preprocessing steps for a clean Tableau integration.

In [ ]:
# import the pandas library
import pandas as pd

# read in the csv file as a pandas dataframe
bikes = pd.read_csv("london_merged.csv")

### **1.1 Structural Overview**
I start by checking the data types and null counts. In the London Bike dataset, the `timestamp` column is often loaded as a generic object (string). To perform time-series analysis in Tableau, I will need to ensure this is handled correctly.

In [4]:
# exploring the data
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


### **1.2 Data Volume & Sample Inspection**
By checking the shape and the first few rows, I can confirm the dataset contains over 17,000 hourly snapshots. 

**Initial Observations:**
- **`cnt`**: Our target variable (number of bike shares).
- **`t1` vs `t2`**: Allows for an interesting "Feel Like" vs "Actual" temperature comparison.
- **`weather_code` & `season`**: Currently stored as floats/integers, which will require semantic mapping to be useful for visualization.

In [5]:
# display the shape of the dataframe
bikes.shape

(17414, 10)

In [6]:
# display the first 5 rows of the dataframe
bikes

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


### **1.3 Categorical Integrity**
I am checking the unique values for `weather_code` and `season` to ensure they align with the official Kaggle metadata documentation. This confirms there are no "ghost" categories or unexpected outliers before I begin the mapping process.

In [7]:
# count the unique values in the weather_code column
bikes.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [8]:
# count the unique values in the season column
bikes.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [9]:
#checking for missing values
bikes.isnull().sum()

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64

In [10]:
# check for duplicate rows
bikes.duplicated().sum()

np.int64(0)

<a id="2"></a>

## **2. Schema Standardization**

Standardizing column names ensures that when the data is loaded into Tableau, the field names are intuitive for non-technical users.

In [11]:
# specifying the column names that I want to use
new_cols_dict ={
    'timestamp':'time',
    'cnt':'count', 
    't1':'temp_real_C',
    't2':'temp_feels_like_C',
    'hum':'humidity_percent',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

# Renaming the columns to the specified column names
bikes.rename(new_cols_dict, axis=1, inplace=True)

<a id="3"></a>
## **3. Feature Engineering & Semantic Mapping**

Raw datasets often use integer encoding for categorical data to save space. However, for analysis, these must be converted into human-readable strings.




### **3.1 Normalization**
I am converting the `humidity` values into a decimal percentage (0 to 1). This aligns the scale with standard meteorological reporting and makes it easier to create "Humidity %" calculations in Tableau.


In [12]:
# changing the humidity values to percentage (i.e. a value between 0 and 1)
bikes.humidity_percent = bikes.humidity_percent / 100

### **3.2 Semantic Decoding**
Using the dataset metadata, I created dictionaries to map:
- **Seasons:** Converting 0-3 into 'Spring', 'Summer', etc.
- **Weather:** Converting specific codes into descriptive labels like 'Broken Clouds' or 'Rain'.

*This ensures that the final dashboard filters and legends are immediately understandable to a business user.*

In [13]:
# creating a season dictionary so that we can map the integers 0-3 to the actual written values
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'autumn',
    '3.0':'winter'
}

# creating a weather dictionary so that we can map the integers to the actual written values
weather_dict = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snowfall'
}

# changing the seasons column data type to string
bikes.season = bikes.season.astype('str')
# mapping the values 0-3 to the actual written seasons
bikes.season = bikes.season.map(season_dict)

# changing the weather column data type to string
bikes.weather = bikes.weather.astype('str')
# mapping the values to the actual written weathers
bikes.weather = bikes.weather.map(weather_dict)

<a id="4"></a>
## **4. Data Export & Verification**

Before exporting, I perform a final `head()` check to ensure the mapping was successful and the data types are correct. 



In [14]:
# checking our dataframe to see if the mappings have worked
bikes.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


The final dataset is exported as an **Excel (.xlsx)** file. By performing these "heavy" transformations in Python rather than Tableau:
1. **Performance is optimized:** Tableau does not need to run complex IF/THEN statements for mapping.
2. **Reproducibility is guaranteed:** The cleaning logic is documented here in code rather than hidden in a dashboard's calculated fields.

In [15]:
# writing the final dataframe to an excel file that we will use in our Tableau visualisations. The file will be the 'london_bikes_final.xlsx' file and the sheet name is 'Data'
bikes.to_excel('london_bikes_final.xlsx', sheet_name='Data')

# 📊 London Bike Rides: Final Dashboard

This dashboard serves as the final visualization layer for a full end-to-end data pipeline. After performing **ETL** on the London Bike Sharing dataset using Python and Pandas, I designed this interactive interface to uncover hidden patterns in urban mobility.

---
#  
**[Dashboard on Tableau ](https://public.tableau.com/shared/XWQGWS5X9)**


---

<p align="center">
  <strong>Final Dashboard</strong><br>
  <br>
  <img src="images/Hero.png" width="80%" title="Full Dashboard Overview">
</p>

<table style="width:80%; margin-left:auto; margin-right:auto; border:none; border-collapse:collapse;">
  <tr>
    <td style="width:50%; border:none; text-align:center; padding:10px;">
       <p><strong>Dynamic Trends</strong></p>
       <img src="images/MovingAverage.png" width="100%" />
    </td>
    <td style="width:50%; border:none; text-align:center; padding:10px;">
       <p><strong>Weather Correlation</strong></p>
       <img src="images/Heatmap.png" width="100%" />
    </td>
  </tr>
  <tr>
    <td style="width:50%; border:none; text-align:center; padding:10px;">
       <p><strong>Weather Distribution</strong></p>
       <img src="images/Weather.png" width="100%" />
    </td>
    <td style="width:50%; border:none; text-align:center; padding:10px;">
       <p><strong>Hourly Demand</strong></p>
       <img src="images/Hour.png" width="100%" />
    </td>
  </tr>
</table>
